<a href="https://colab.research.google.com/github/JericCantos/TIL/blob/master/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Load Data

In [1]:
import pandas as pd
import requests
import io
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Build the direct download URL from a file ID
def make_drive_url(file_id):
    return f"https://drive.google.com/uc?id={file_id}"

# Helper function to load a CSV from a direct URL
def load_csv_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises an error if the request fails
    return pd.read_csv(io.StringIO(response.text))

# Dictionary of file IDs for clarity
file_ids = {
    "holiday_events": "1RMjSuqHXHTwAw_PGD5XVjhA3agaAGHDH",
    "items": "1ogMRixVhNY6XOJtIRtkRllyOyzw1nqya",
    "oil": "1Q59vk2v4WQ-Rpc9t2nqHcsZM3QWGFje_",
    "stores": "1Ei0MUXmNhmOcmrlPad8oklnFEDM95cDi",
    "train": "1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv",
    "transactions": "1PW5LnAEAiL43fI5CRDn_h6pgDG5rtBW_"
}

# Load each CSV using the helper functions
df_holiday_events = load_csv_from_url(make_drive_url(file_ids["holiday_events"]))
df_items          = load_csv_from_url(make_drive_url(file_ids["items"]))
df_oil            = load_csv_from_url(make_drive_url(file_ids["oil"]))
df_stores         = load_csv_from_url(make_drive_url(file_ids["stores"]))
# df_train          = load_csv_from_url(make_drive_url(file_ids["train"])) we dont read it as the file is too big and wont work this way
df_transactions   = load_csv_from_url(make_drive_url(file_ids["transactions"]))


## Loading df_train

filter only to stores in "Guayas"

In [5]:
!pip install -U gdown

In [6]:
import gdown

# Use our existing function to build the download URL
train_url = make_drive_url(file_ids["train"])

# Download the file using gdown
gdown.download(train_url, "train.csv", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv
From (redirected): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv&confirm=t&uuid=1f63ceb6-ec09-489d-8dae-46ce2389c255
To: /content/train.csv
100%|██████████| 5.00G/5.00G [01:09<00:00, 71.8MB/s]


'train.csv'

In [7]:
# select only stores in 'Guayas' state
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()


# read the imported. data in chunks
chunk_size = 10**6  # 1 million rows at a time
filtered_chunks = []

for chunk in pd.read_csv("train.csv", chunksize=chunk_size):
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(chunk_filtered)
    del chunk  # Free up memory

# combine all chunks into one dataframe
df_train = pd.concat(filtered_chunks, ignore_index=True)

# down-sample to 2 million rows
df_train = df_train.sample(n=2_000_000).reset_index(drop=True)

/tmp/ipython-input-7-946473318.py:9: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("train.csv", chunksize=chunk_size):


### Filter by Product Family
- keep only the three biggest product families (measured by how many unique items each family contains)

In [8]:
df_items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [12]:
df_items['family'].value_counts()
# top three families are: GROCERY I,  BEVERAGES, and CLEANING

,count
family,
GROCERY I,1334
BEVERAGES,613
CLEANING,446
PRODUCE,306
DAIRY,242
PERSONAL CARE,153
BREAD/BAKERY,134
HOME CARE,108
DELI,91


In [16]:
# join df_train with df_items then filter only top 3

top_3_families = ['GROCERY I',  'BEVERAGES', 'CLEANING']

df_train_family = pd.merge(
    df_train,
    df_items,
    on = 'item_nbr',
    how = 'inner'

    )

df_train_family.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable
0,29340652,2014-08-19,51,417835,1.000,False,GROCERY I,1028,0
1,1324310,2013-02-03,32,155500,18.000,NaN,GROCERY I,1014,0
2,88558245,2016-08-20,24,1239862,2.597,False,POULTRY,2420,1
3,111800519,2017-04-08,24,1986500,1.000,False,HOME CARE,3108,0
4,56649285,2015-09-15,28,527757,2.000,False,GROCERY I,1032,0


In [17]:
print ('original dataset length:', len(df_train),
       '\nwith families length:', len(df_train_family) )
df_train = df_train_family[df_train_family['family'].isin(top_3_families)]

print ('filtered top 3 length:', len(df_train))

original dataset length: 2000000 
with families length: 2000000
filtered top 3 length: 1284286


In [18]:
df_train['family'].unique()

array(['GROCERY I', 'BEVERAGES', 'CLEANING'], dtype=object)

### Save Pickle

In [20]:
# 1. Mount Drive (one-time click to authorize)
from google.colab import drive
drive.mount('/content/drive')

# 2. Pick a folder inside Drive (create it if it doesn’t exist)
save_path = "/content/drive/MyDrive/Masterschool_Time_Series/df_train_clean.pkl"


# 3. SAVE the cleaned DataFrame
df_train.to_pickle(save_path)
print(f"Saved to: {save_path}")



Mounted at /content/drive
Saved to: /content/drive/MyDrive/Masterschool_Time_Series/df_train_clean.pkl


### Load Pickle



In [ ]:
# -----------------------------------------------
# Uncomment and Run this to reload dataframe
# -----------------------------------------------

'''
from google.colab import drive
drive.mount('/content/drive')              # re-mount
load_path = "/content/drive/MyDrive/Masterschool_Time_Series/df_train_clean.pkl" #example of path, use the same as above
df_train = pd.read_pickle(load_path)
print("DataFrame reloaded, ready to go!")
'''